In this code we calculate the Jordan decomposition of a Matrices $A$ (whose characteristic polynomial can be factorized symbolically) from derivatives of the adjugate $B(Z) = z-A$ (see below for more information), in the sense that
\begin{equation}
A = \sum_{i=1}^m \left( N_i + \lambda_i \right) P_i\,.
\end{equation}
This decomposition is not actually the Jordan decomposition, because it doesn't compute the Jordan chains, however, it computes projections and nilpotent matrix, that can be used to obtain matrix functions as shown in example 3.

In [24]:
import sympy as sp

The Jordan canonical form can be announed as follows.

Let $A$  be a $n\times n$ complex matrix, let $\{\lambda_i\}_{1\leq i\leq m}$ denote the $m$ different eigenvalues of A with corresponding algebraic multiplicities $\{n_i\}_{1\leq i\leq m}$. Then, the characteristic polynomial is 
$$
p(z)=\prod_{i=1}^m (z-\lambda_i)^{n_i}.
$$
We can define $\{W_i\}_{1\leq i\leq m}$, the invariant sub-spaces $W_{\lambda_i}=W_i = \mbox{ker}(A-\lambda_i)^{n_i}$ and projections $P_i$ onto the spaces $W_i$, then exist a matrix $V$ such that $A = V J V^{-1}$, where $J$ is a block diagonal matrix, whose diagonal blocks are square and of the form
\begin{align}
    J(\lambda)_{rr} &= 
  \begin{pmatrix}
    \lambda & 1 & 0 & 0 & ... & 0 & 0\\
     0 &  \lambda & 1 & 0 & ...& 0 & 0\\
     . & . & . & . & .& . & . \\
    . & . & . & . & .& . & . \\
    0 &  0 & 0 & 0 & ... & \lambda & 1 \\
    0 &  0 & 0 & 0 & ...& 0 & \lambda
  \end{pmatrix}\, ,
\end{align}
for $1\leq r\leq k$ (in general, $k$ depends on $A$), $\lambda$ is an eigenvalue of $A$ and the matrix $V$ contains the generalized eigenvectors associated with the corresponding Jordan blocks, which in turn belong to the corresponding spaces $W_{\lambda}$. There may be more than one Jordan block associated with the same eigenvalue.

# Theorem

Given a $n\times n$ complex matrix $A$, let $\{\lambda_i\}_{1\leq i\leq m}$,$\{n_i\}_{1\leq i\leq m}$, $\{W_i\}_{1\leq i\leq m}$ and $\{P_i\}_{1\leq i\leq m}$ be defined as above, then $\mathbb{C}^n = \bigoplus W_i$ and there exist $N_i$, nilpotent operators of degree $d_i \leq n_i$, $1\leq i\leq m$,  such that $P_i N_i 
 = N_i P_i$ for all $i$ (we write the commutator $[N_i,P_i]=0$, for short), we have
\begin{equation}
A = \sum_{i=1}^m \left( N_i + \lambda_i \right) P_i
\end{equation}
and the projectors satisfy $ \sum_{i=1}^m P_i = \mathbb{1}$ and 
\begin{align} 
    P_i P_j &= \delta_{ij} P_i\quad \text{for all $1\leq i,j\leq m$}.
\end{align}

#Theorem

Let $B(z)=\mbox{Adj}(z-A)$ and $q_i(z) = p(z)/(z-\lambda_i)^{n_i} =\prod_{j\not =i }^m (z-\lambda_j)^{n_j}$, i.e, the multiplication of the other factors of $p$ with $q_i(\lambda_i) \neq 0$. Then
\begin{equation}
    P_i=\frac{1}{(n_i-1)!}\frac{d^{n_i-1} }{dz^{n_i-1}} \left( \frac{B(z)}{q_i(z)}\right)\Bigr|_{\lambda_i}\,.
\end{equation}
Moreover, if $n_i\geq2$, then
\begin{align}
    N_i P_i &= \frac{1}{(n_i-2)!}\frac{d^{n_i-2} }{dz^{n_i-2}} \left( \frac{B(z)}{q_i(z)}\right)\Bigr|_{\lambda_i}\,.
\end{align}
In addition, 
\begin{equation}
    \frac{d^{n_i-1} B(z)}{dz^{n_i-1}}\Bigr|_{\lambda_i} = (n_i-1)! \, \textbf{   } q_i\left(N_i + \lambda_i\right)  P_i.
\end{equation}
Thus, in the case that the geometric and algebraic dimensions of $\lambda_i$ are the same (as happens when $A$ is diagonalizable), then $N_i = 0$ and $P_i$ can be obtained as
\begin{equation}
    \frac{d^{n_i-1} B(z)}{dz^{n_i-1}}\Bigr|_{\lambda_i} = (n_i-1)! \, \textbf{   } q_i\left(\lambda_i\right)  P_i.
\end{equation}
You can check these theorems in https://arxiv.org/abs/2303.09953

#Functions

In [25]:
#Given a matrix A, returns z I - A
def LambdaMinusA(A):
    z = sp.Symbol('z')
    n,m = A.shape
    return z*sp.eye(n) - A,z

In [26]:
#Given the characteristic polynomial P_z, a root ri and its degree ni,
#returns q_i(z) = P_z/(z-r)^ni, which is a coprime polynomial to (z-ri)
def q_i(P_z,r,ni):
  #return sp.simplify(P_z/((z-r)**ni))
  Factors = sp.roots(P_z)
  roots = list(Factors.keys())
  q = 1
  z = sp.Symbol('z')
  for root in roots:
    if(root != r):
      q*=(z-root)**(Factors[root])
  return q

In [27]:
#Given a matrix A, returns Rz = z-A,Bz = adj(z-A),P_z = det(Rz)
def Matrices(A):
  Rz,z = LambdaMinusA(A);
  Bz = Rz.adjugate()
  P_z = sp.det(Rz)
  PzFactors = sp.factor(P_z)
  return Rz,Bz,P_z,PzFactors,z

In [28]:
#if flag = 0, then returns the Riesz Projection P_i
#if flog = 1, returns (N_i + r_i ) P_i = A*P_i , N_i nilpotent
def RietzIntegral(Bz,P_z,ri,ni,flag):
  z = sp.Symbol('z')
  flag1 = flag;
  ndev = ni-1;
  if(ri == 0 and flag==1):
    flag1 = 0;
    ndev = ni-2;
  BzUq_i = sp.simplify((z**flag1)*Bz/q_i(P_z,ri,ni))
  B = sp.diff(BzUq_i,z,ndev)
  B = sp.simplify(B/sp.factorial(ndev))
  return B,B.subs(z,ri)

In [29]:
#Check that in the normal case (unitary similar to a diagonal matrix), the 0<=s<n-1 derivative of Bz evaulated in r_i, is zero
def check_order(A,Bz,roots,Factorization):
  if(not A.is_symmetric()):
    print("Your matrix is not symmetryc")
  else:
    z = sp.Symbol('z')
    n_A,n_A = A.shape
    checking = True
    for r in roots:
      degree = Factorization[r]
      for s in range(0,degree-1):
        Bdev = sp.diff(Bz,z,s)
        Bdevi = Bdev.subs(z,r)
        if(Bdevi != sp.zeros(n_A)):
          checking = False
          print("Oh oh, Something went wrong")
          break
    if(checking == True):
      print("\n\nTest approved, all s derivatives (0<=s<= n_i-1) of B_z evaluated in r_i are zero\n\n")

In [30]:
#check if A = Sum (ri + Ni) Pi, where Ni,ri and Pi are obtained with the code
def check_decomposition(A,PrPlusN,roots):
  checking = True
  B = 0*A
  for r in roots:
    B += PrPlusN[r]
  B = sp.simplify(B)
  if(A!=B):
    checking = False
    print("Oh,oh, somethig went grong")
  else:
    print("\n\nTest approved, correct decomposition")
  return A-B

#check if I = Sum Pi, where Pi are obtained with the code
def check_sum(A,Peval,roots):
  n,m = A.shape
  B = sp.eye(n)
  C = sp.zeros(n)
  for r in roots:
    C += Peval[r]
  C = sp.simplify(C)
  if(C!=B):
    print("Oh,oh, somethig went grong with Sum P_i = I")
  else:
    print("\n\nTest approved, Sum P_i = I")
  return C


#check if NiPi is nilpotent
def check_Nilpotent(A,Nilpotents,roots,Factorization):
  checking = True
  n,m = A.shape
  C = sp.zeros(n)
  D = sp.zeros(n,1)
  for r in roots:
    degree = degree = Factorization[r]
    B = Nilpotents[r]**degree
    B = sp.simplify(B)
    D = D.col_insert(0,B)
    if(C!=B):
      print("Oh,oh, somethig went grong with ( Ni Pi )^n_i = 0")
      checking = False
  if(checking):
    print("\n\nTest approved, All N_i P_i are nilpotents")
  return D

#check if the P_i are projections, thta is, P_i*P_i = P_i
def check_projection(A,Peval,roots):
  checking_projections = True
  n,m = A.shape
  for r in roots:
    Pr = Peval[r]
    if(not (sp.simplify(Pr*Pr-Pr)==sp.zeros(n))):
      checking_projections = False
      print("\nSome Pr is not a projection. Something went wrong!")
  if(check_projection):
    print("\nTest Approved, All P_i are projections!")

In [31]:
#if print_flag = 1, then P_i and N_i will be printed
#Given a matrix A, returns Rz,Bz,P_z,Factors,z,P and Peval
def GetProjections(A,print_flag):
  n,m = A.shape
  Rz,Bz,P_z,PzFactors,z = Matrices(A)
  Factorization = sp.roots(P_z)
  roots = Factorization.keys()
  P = {}
  Peval = {}
  Nilpotents = {}
  PrPlusN = {}
  checking_projections = True
  for r in roots:
    Pz,Pr = RietzIntegral(Bz,P_z,r,Factorization[r],0)
    PzN,PrN = RietzIntegral(Bz,P_z,r,Factorization[r],1)
    P[r] = Pz
    Peval[r] = Pr
    Nr = PrN - r*Pr
    Nilpotents[r] = Nr
    PrPlusN[r] = PrN
    if(print_flag==1):
      print("\nroot_i:\t",r,"\t Projection_i:\n")
      #print("\nroot_i:\t",r,"\t (r_i + N_i) Projection_i:\n")
      sp.pprint(sp.simplify(Pr))
      print("\nroot_i:\t",r,"\t Nilpotent_i:\n")
      sp.pprint(sp.simplify(Nr))
    #sp.pprint(sp.simplify(Pr))
  return Rz,Bz,P_z,PzFactors,P,Peval,Nilpotents,PrPlusN,z,roots,Factorization

#Exmaple 1

In this example, we are going to calculate the spectral decomposition of a symmetric matrix A, that is,
\begin{align}
A = \sum \lambda_i P_i\,,
\end{align}
where $\lambda_i$ belongs to the spectrum of $A$ and $P_i$ is the orthogonal projections onto the eigen-spaces.









In [32]:
#M= sp.Matrix([[1,-1,1],[-1,1,-1],[1,-1,1]]);
M = sp.Matrix([[3,2,1],[2,3,1],[1,1,4]])
M

Matrix([
[3, 2, 1],
[2, 3, 1],
[1, 1, 4]])

In [33]:
flag_checking = True

In [34]:
#If you run this cell, at the end it should appear "Test approved, correct decomposition" and 
# "Test approved, all s derivatives (0<=s<= n_i-1) of B_z evaluated in r_i are zero"
Rz,Bz,P_z,Factors,P,Peval,Nilpotents,PrPlusN,z,roots,Factorization = GetProjections(M,1)
if(flag_checking):
  print("Characteristic polynomial:")
  print(Factors)
  D=check_decomposition(M,PrPlusN,roots)
  I=check_sum(M,Peval,roots)
  check_order(M,Bz,roots,Factorization)
  N=check_Nilpotent(M,Nilpotents,roots,Factorization)
  check_projection(M,Peval,roots)


root_i:	 6 	 Projection_i:

⎡1/3  1/3  1/3⎤
⎢             ⎥
⎢1/3  1/3  1/3⎥
⎢             ⎥
⎣1/3  1/3  1/3⎦

root_i:	 6 	 Nilpotent_i:

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

root_i:	 3 	 Projection_i:

⎡1/6   1/6   -1/3⎤
⎢                ⎥
⎢1/6   1/6   -1/3⎥
⎢                ⎥
⎣-1/3  -1/3  2/3 ⎦

root_i:	 3 	 Nilpotent_i:

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

root_i:	 1 	 Projection_i:

⎡1/2   -1/2  0⎤
⎢             ⎥
⎢-1/2  1/2   0⎥
⎢             ⎥
⎣ 0     0    0⎦

root_i:	 1 	 Nilpotent_i:

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦
Characteristic polynomial:
(z - 6)*(z - 3)*(z - 1)


Test approved, correct decomposition


Test approved, Sum P_i = I


Test approved, all s derivatives (0<=s<= n_i-1) of B_z evaluated in r_i are zero




Test approved, All N_i P_i are nilpotents

Test Approved, All P_i are projections!


#Example 2

In this example, we are going to calculate the Jordan decomposition of a general matrix A with charcateristic polynomial
\begin{align}
p(z) = \Pi (z-\lambda_i)^{n_i}\,.
\end{align}
Thus, 
\begin{align}
A = \sum \left(\lambda_i + N_i \right)P_i\,
\end{align}
where $\lambda_i$ belongs to the spectrum of $A$, $P_i$ are projections onto the $\mbox{Ker}\left((A-\lambda_i)^{n_i}\right)$ and $N_i$ is a nilpotent matrix of degree $d_i \leq n_i$.

In [35]:
#Ms = [[1,-1,1],[-1,1,-1],[1,-1,1]]
#Ms = [[1,1,0,0],[0,1,0,0],[0,0,-2,0],[0,0,0,4]]
#Ms = [[3,-1,1],[2,1,0],[1,-1,2]]
#Ms = [[1,2,1,1,0],[0,1,0,0,0],[0,0,1,1,0],[0,0,0,1,0],[0,0,0,0,1]]
#Ms = [[0,1,0,0],[11,6,-4,-4],[22,15,-8,-9],[-3,-2,1,2]]
Ms = [[-3,1,0,1,1],[-3,1,0,1,1],[-4,1,0,2,1],[-3,1,0,1,1],[-4,1,0,1,2]]
M = sp.Matrix(Ms)
M

Matrix([
[-3, 1, 0, 1, 1],
[-3, 1, 0, 1, 1],
[-4, 1, 0, 2, 1],
[-3, 1, 0, 1, 1],
[-4, 1, 0, 1, 2]])

In [36]:
flag_checking = True

In [37]:
#If you run this cell, at the end it should appear "Test approved, correct decomposition" and 
# "Test approved, all s derivatives (0<=s<= n_i-1) of B_z evaluated in r_i are zero"
Rz,Bz,P_z,Factors,P,Peval,Nilpotents,PrPlusN,z,roots,Factorization = GetProjections(M,1)
if(flag_checking):
  print("Characteristic polynomial:")
  print(Factors)
  D=check_decomposition(M,PrPlusN,roots)
  I=check_sum(M,Peval,roots)
  check_order(M,Bz,roots,Factorization)
  N=check_Nilpotent(M,Nilpotents,roots,Factorization)
  check_projection(M,Peval,roots)


root_i:	 1 	 Projection_i:

⎡-1  0  0  0  1⎤
⎢              ⎥
⎢-1  0  0  0  1⎥
⎢              ⎥
⎢-1  0  0  0  1⎥
⎢              ⎥
⎢-1  0  0  0  1⎥
⎢              ⎥
⎣-2  0  0  0  2⎦

root_i:	 1 	 Nilpotent_i:

⎡0  0  0  0  0⎤
⎢             ⎥
⎢0  0  0  0  0⎥
⎢             ⎥
⎢0  0  0  0  0⎥
⎢             ⎥
⎢0  0  0  0  0⎥
⎢             ⎥
⎣0  0  0  0  0⎦

root_i:	 0 	 Projection_i:

⎡2  0  0  0  -1⎤
⎢              ⎥
⎢1  1  0  0  -1⎥
⎢              ⎥
⎢1  0  1  0  -1⎥
⎢              ⎥
⎢1  0  0  1  -1⎥
⎢              ⎥
⎣2  0  0  0  -1⎦

root_i:	 0 	 Nilpotent_i:

⎡-2  1  0  1  0⎤
⎢              ⎥
⎢-2  1  0  1  0⎥
⎢              ⎥
⎢-3  1  0  2  0⎥
⎢              ⎥
⎢-2  1  0  1  0⎥
⎢              ⎥
⎣-2  1  0  1  0⎦
Characteristic polynomial:
z**4*(z - 1)


Test approved, correct decomposition


Test approved, Sum P_i = I
Your matrix is not symmetryc


Test approved, All N_i P_i are nilpotents

Test Approved, All P_i are projections!


#Example 3

As in example 2, we are going to calculate the Jordan decomposition of a general matrix A with charcateristic polynomial
\begin{align}
p(z) = \Pi (z-\lambda_i)^{n_i}\,.
\end{align}
Thus, 
\begin{align}
A = \sum \left(\lambda_i + N_i \right)P_i\,
\end{align}
where $\lambda_i$ belongs to the spectrum of $A$, $P_i$ are projections onto the $\mbox{Ker}\left((A-\lambda_i)^{n_i}\right)$ and $N_i$ is a nilpotent matrix of degree $d_i \leq n_i$.
Later, we compute $\mbox{exp}(A) = \mathbb{1} + A + A^2/2! + ...$ taking advantage of the property of nilpotent matrices. Since, for an analytic finction $f(z)$, (see https://arxiv.org/abs/2303.09953)
\begin{align*}
f(A) &= \sum_i f(\lambda_i + N_i) P_i\,\\
f(\lambda_r + N_r) P_r &= \sum_{l=0}^{\infty} \frac{f^{(l)}(\lambda_r)}{l!}N_r^l P_r\,.
\end{align*}

In [38]:
#Ms = [[1,-1,1],[-1,1,-1],[1,-1,1]]
#Ms = [[1,1,0,0],[0,1,0,0],[0,0,-2,0],[0,0,0,4]]
#Ms = [[3,-1,1],[2,1,0],[1,-1,2]]
#Ms = [[1,2,1,1,0],[0,1,0,0,0],[0,0,1,1,0],[0,0,0,1,0],[0,0,0,0,1]]
Ms = [[0,1,0,0],[11,6,-4,-4],[22,15,-8,-9],[-3,-2,1,2]]
#Ms = [[-3,1,0,1,1],[-3,1,0,1,1],[-4,1,0,2,1],[-3,1,0,1,1],[-4,1,0,1,2]]
M = sp.Matrix(Ms)
M

Matrix([
[ 0,  1,  0,  0],
[11,  6, -4, -4],
[22, 15, -8, -9],
[-3, -2,  1,  2]])

In [39]:
flag_checking = False

In [40]:
Rz,Bz,P_z,Factors,P,Peval,Nilpotents,PrPlusN,z,roots,Factorization = GetProjections(M,0)
if(flag_checking):
  D=check_decomposition(M,PrPlusN,roots)
  I=check_sum(M,Peval,roots)
  check_order(M,Bz,roots,Factorization)
  N=check_Nilpotent(M,Nilpotents,roots,Factorization)
  check_projection(M,Peval,roots)

In [41]:
#characteristic polynomial
Factors

(z - 1)**2*(z + 1)**2

In [42]:
P1 = Peval[1]; N1 = Nilpotents[1]; #associated with lambda = 1
P2 = Peval[-1]; N2 = Nilpotents[-1]; #associated with lambda = -1

Since,
\begin{align*}
f(A) &= \sum_i f(\lambda_i + N_i) P_i\,\\
f(\lambda_r + N_r) P_r &= \sum_{l=0}^{\infty} \frac{f^{(l)}(\lambda_r)}{l!}N_r^l P_r\,.
\end{align*}
In particular, for the matrix chosen, the algebraic dimension of the roots are not greater than 2, then
\begin{align*}
f(A) &= (f(1)P_1 + f'(1) P_1) + (f(-1)P_2 + f'(-1) P_2)\,.
\end{align*}
So, for $f(z) = \mbox{exp}(z)$,
\begin{align*}
\mbox{exp}(A) &= (\mbox{exp}(1)P_1 + \mbox{exp}(1) P_1) + (\mbox{exp}(-1)P_2 + \mbox{exp}(-1) P_2)\,.
\end{align*}

In [43]:
expA_obtained = sp.exp(1)*(P1+N1) + sp.exp(-1)*(P2+N2)
expA_obtained = sp.simplify(expA_obtained)
expA_obtained

Matrix([
[  (-7 + 3*exp(2))*exp(-1),   (-5 + 2*exp(2))*exp(-1),   (3 - exp(2))*exp(-1),   (3 - exp(2))*exp(-1)],
[   (2 + 3*exp(2))*exp(-1),                 4*cosh(1),             -2*cosh(1),             -2*cosh(1)],
[(-17 + 15*exp(2))*exp(-1), (-11 + 10*exp(2))*exp(-1), (7 - 5*exp(2))*exp(-1), (7 - 6*exp(2))*exp(-1)],
[                     -3*E,                      -2*E,                      E,                    2*E]])

Compare with $\exp(A)$ computed with a sympy function

In [44]:
expSP = sp.exp(M)
expSP = sp.simplify(expSP)
expSP

Matrix([
[  (-7 + 3*exp(2))*exp(-1),   (-5 + 2*exp(2))*exp(-1),   (3 - exp(2))*exp(-1),   (3 - exp(2))*exp(-1)],
[   (2 + 3*exp(2))*exp(-1),                 4*cosh(1),             -2*cosh(1),             -2*cosh(1)],
[(-17 + 15*exp(2))*exp(-1), (-11 + 10*exp(2))*exp(-1), (7 - 5*exp(2))*exp(-1), (7 - 6*exp(2))*exp(-1)],
[                     -3*E,                      -2*E,                      E,                    2*E]])